In [1]:
from pyscal.core import System
from pyscal.crystal_structures import Structure
from pyscal.operations.symmetry import Symmetry
import pyscal_rdf.properties as prp
from pyscal_rdf.json_io import write_file
import numpy as np

In [2]:
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    #Unit cell properties
    info["LatticeParameter"] = sys.atoms._lattice_constant
    
    #Simulation cell properties
    info["Coordination"] = prp.get_coordination(sys)
    symbol, number = prp.get_space_group(sys)
    info["SpaceGroupSymbol"] = symbol
    info["SpaceGroupNumber"] = number
    
    #needs to be defined
    if sys.atoms.species[0] is not None:
        info["Element"] = np.unique(sys.atoms.species)
        info["Occupancy"] = sys.atoms.species
    else:
        info["Element"] = np.unique(sys.atoms.types)
        info["Occupancy"] = sys.atoms.types
    
    info["BravaisLattice"] = sys.atoms._lattice 
    info["CellVolume"] = sys.volume
    info["NumberOfAtoms"] = sys.natoms
    
    box_dims = sys.box_dimensions
    info["SimulationCellLengthX"] = box_dims[0]
    info["SimulationCellLengthY"] = box_dims[1]
    info["SimulationCellLengthZ"] = box_dims[2]

    info["SimulationCellVectorA"] = sys.box[0]
    info["SimulationCellVectorB"] = sys.box[1]
    info["SimulationCellVectorC"] = sys.box[2]
    
    info["SimulationCellAngleAlpha"] = prp.get_angle(sys.box[0], sys.box[1])
    info["SimulationCellAngleBeta"] = prp.get_angle(sys.box[1], sys.box[2])
    info["SimulationCellAngleGamma"] = prp.get_angle(sys.box[2], sys.box[0])    
    
    return info

## POSCAR

In [3]:
sys = System("al_data/Al.poscar", format="poscar")

In [4]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))

## LAMMPS - dump

In [5]:
sys = System("al_data/Al.dump")

In [6]:
convert_to_dict(sys)
write_file("dump2", convert_to_dict(sys))

## CIF/ ASE

In [7]:
from ase.io import read

In [8]:
aseobj = read("al_data/Al.cif", format="cif")

In [9]:
sys = System(aseobj, format="ase")

In [10]:
convert_to_dict(sys)
write_file("dump3", convert_to_dict(sys))

In [11]:
data = convert_to_dict(sys)

## Creating structure from pyscal

In [18]:
struct = Structure()

In [19]:
sys = struct.element.Ag()
sym = Symmetry(sys)
sym.calculate()

Ag


In [22]:
sym.output.international_space_group_number, sym.output.international_symbol, sym.output.point_group

(225, 'Fm-3m', 'm-3m')

In [24]:
sym.output.international_symbol?

Type:        property
String form: <property object at 0x7f50fb4f1850>
Docstring:  
The (full) Hermann–Mauguin notation of space group type  
Unit: None
Default: None
url: https://www.chemeurope.com/en/encyclopedia/Hermann-Mauguin_notation.html
Optional: False
type: None

In [13]:
sys = struct.lattice.b2(lattice_constant=4, element=["Ni", "Al"])

['Ni', 'Al']


In [14]:
convert_to_dict(sys)

{'latticeparameter': 4,
 'Coordination': [14, 14],
 'SpaceGroup': None,
 'Element': array(['Al', 'Ni'], dtype='<U2'),
 'Occupancy': ['Ni', 'Al'],
 'Lattice': 'b2',
 'CellVolume': 1728.0,
 'NumberOfAtoms': 2,
 'X_AxisCoordinate': 4.0,
 'Y_AxisCoordinate': 4.0,
 'Z_AxisCoordinate': 4.0,
 'FirstAxisComponent': [4.0, 0, 0],
 'SecondAxisComponent': [0, 4.0, 0],
 'ThirdAxisComponent': [0, 0, 4.0],
 'LatticeParameterLengthA': 4.0,
 'LatticeParameterLengthB': 4.0,
 'LatticeParameterLengthC': 4.0,
 'LatticeParameterAngleAlpha': 90.0,
 'LatticeParameterAngleBeta': 90.0,
 'LatticeParameterAngleGamma': 90.0}

In [15]:
sys = struct.element.Si()

Si


In [16]:
convert_to_dict(sys)

{'latticeparameter': 5.43,
 'Coordination': [4, 4, 4, 4, 4, 4, 4, 4],
 'SpaceGroup': None,
 'Element': array(['Si'], dtype='<U2'),
 'Occupancy': ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si'],
 'Lattice': 'diamond',
 'CellVolume': 4322.781188999999,
 'NumberOfAtoms': 8,
 'X_AxisCoordinate': 5.43,
 'Y_AxisCoordinate': 5.43,
 'Z_AxisCoordinate': 5.43,
 'FirstAxisComponent': [5.43, 0, 0],
 'SecondAxisComponent': [0, 5.43, 0],
 'ThirdAxisComponent': [0, 0, 5.43],
 'LatticeParameterLengthA': 5.43,
 'LatticeParameterLengthB': 5.43,
 'LatticeParameterLengthC': 5.43,
 'LatticeParameterAngleAlpha': 90.0,
 'LatticeParameterAngleBeta': 90.0,
 'LatticeParameterAngleGamma': 90.0}

## TODO: pyscal side

- ~~Take a list of elements (optionally)~~ 
- ~~Lattice constant is not multipled to structure creation~~
- Units should always be Angstroms
- ~~Lattice constant is wrong in the above example ; should be 3 - but 9 is used~~
- ~~Always use Cartesian coordinates~~
- Space group

RDFLIB

In [ ]:
from rdflib import FOAF, Graph, Literal, Namespace, XSD, RDF, RDFS, URIRef, BNode, SDO
from rdflib import term

In [ ]:
CSO = Namespace("https://purls.helmholtz-metadaten.de/disos/cso/")
MDO = Namespace("https://w3id.org/mdo/structure/1.1/")

In [ ]:
g = Graph()
#g.bind("cso", CSO)
#g.bind("mdo", MDO)

In [ ]:
struct1 = BNode()
struct2 = BNode()

In [ ]:
g.add((struct1, RDF.type, MDO.Structure))
g.add((struct2, RDF.type, MDO.Structure))
g.add((struct1, FOAF.name, Literal("struct1")))
g.add((struct2, FOAF.name, Literal("struct2")))


In [ ]:
print(g.serialize(format="json-ld"))

In [ ]:
MDO.Structure

In [ ]:
Coordinate_Vector = URIRef("https://w3id.org/mdo/structure/CoordinateVector")
Lattice = URIRef("https://purls.helmholtz-metadaten.de/disos/cso#Lattice")
Structure

In [ ]:

g.add((Coordinate_Vector, MDO.X_axisCoordinate, Literal(data["X_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Y_axisCoordinate, Literal(data["Y_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Z_axisCoordinate, Literal(data["Z_AxisCoordinate"])))

In [ ]:
with open("dump-ld.json", "w") as fout:
    fout.write(g.serialize(format="json-ld"))

In [ ]:
print(g.serialize(format="json-ld"))

- Map the terms (see above)
- Terms for type annotations
- Use BNode to add multiple structures
- Add visualisation
- Space group calculation
- try saving to database
- Query the graph (range queries possible?)
- Make a package out of it
- Extend....